# 🌤 Weather Temperature

### Libraries

* Let's import the usual libraries:

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
import tensorflow as tf

* Manipulating temporal data is tricky, let's also import 📚 [`typing`](https://docs.python.org/3/library/typing.html) to check the types of variables we will be dealing with in our Python functions:

In [ ]:
from typing import Dict, List, Tuple, Sequence

# (0) The weather temperature challenge

## (0.0) Introduction

🧑🏻‍🏫 **Goals:**
- Prepare a dataset to be fed into a Recurrent Neural Network
- Develop a better understanding of Time Series

❗️ **Warning/Disclaimer**:
- This challenge is truly designed to help you understand **how to deal with temporal data**, using an LSTM architecture as a _tool_, not to focus on the different gates of the LSTM or designing the "best" recurrent network.

🎯 **ML target**:
* In this challenge, we want to **predict the temperature in the next 3, 6, 9, 12... hours**... 
* ...based on a sequence of weather features such as the _past temperature_, the _atmospheric pressure_, the _humidity_, etc..

## (0.1) The weather dataset

### (0.1.1) Loading the dataset

🌤 This challenge uses a [**weather time series dataset**](https://www.bgc-jena.mpg.de/wetter/) recorded by the [**Max-Planck-Institute for Biogeochemistry**](https://www.bgc-jena.mpg.de/index.php/Main/HomePage). This dataset contains $14$ different features such as _air temperature_, _atmospheric pressure_ and _humidity_ that were collected starting in 2003 every 10 minutes (~ 420k rows). But for efficiency, you will use "only" the data collected between 2009 and 2016 every three hours. Indeed, this time interval seems reasonable to observe the evolution of the temperature throughout a given day.

🛠 We've already performed the following feature engineering steps for you:
- taking every $18$th record to focus on predictions every three hours $ ( 18 =  \frac{6 records}{hour} \times 3 hours)$
- replacing absurd values
- _wind_: computing the wind directions as wind vectors with coordinates (`Wx`, `Wy`)
- computing _the daily and yearly periodicities_, stored through (`Day sin`, `Day cos`) and (`Year sin`, `Year cos`)

##### (Keep this section for later) Downloading the original dataset and engineer the features manually

_(Toggle the section to hide it for the moment)_

🥋 If you want to practice your feature engineering skills, feel free to download the original dataset and work on it (but not today, there are many questions and reading sections to cover 😉)

In [ ]:
#########################################################
# Uncomment later -not today- to download the original  #
# dataset and try to perform the features engineering   #
# by yourself                                           #
#########################################################


zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

raw = pd.read_csv(csv_path)

print(f"raw.shape = {raw.shape}")
display(raw.head())

In [ ]:
# Inspecting each feature to detect their type and null values
raw.info()

In [ ]:
# Slice [start:stop:step], 
# starting from index 5, take every 6th record
# to get only hourly records

pass  # YOUR CODE HERE

In [ ]:
# Convert the "Date Time" column to a datetime format

pass  # YOUR CODE HERE

In [ ]:
# Describe the dataset

pass  # YOUR CODE HERE

In [ ]:
## Fixing the wv

pass  # YOUR CODE HERE

## Fixing the max wv

pass  # YOUR CODE HERE

In [ ]:
# Wind : Wd degrees from 0 to 360 egrees
# Angles do not make good models inputs because 0 and 360 should be "close"

print('-'*50)
print("Working with angles...")

plt.hist2d(df['wd (deg)'], df['wv (m/s)'], 
           bins=(50, 50), 
           vmax=400)
plt.colorbar()
plt.xlabel('Wind Direction [deg]')
plt.ylabel('Wind Velocity [m/s]')
plt.show()

# It is much easier for the model to interpret
# the wind direction and the wind velocity through a vector


# Convert degrees to radians and store the values into wd_rad
pass  # YOUR CODE HERE

# Calculate the wind x and y components and store then in two new columns
# `Wx` and `Wy`
pass  # YOUR CODE HERE

# Calculate the max wind x and y components and store then in two new columns
# `max Wx` and `max Wy`
pass  # YOUR CODE HERE

print('-'*50)
print("Working with wind vectors")

plt.hist2d(df['Wx'], df['Wy'], 
           bins=(50, 50), vmax=400)
plt.colorbar()
plt.xlabel('Wind X [m/s]')
plt.ylabel('Wind Y [m/s]')
ax = plt.gca()
ax.axis('tight')
plt.show()

# $CHALLENGIFY_END

In [ ]:
# Similarly to the wind direction, the time in seconds is not a useful model input
# The weather dataset has clear daily and yearly periodicities
# Using sine and cosine functions, we can compute:
# - the time of the day
# - the time of the year

pass  # YOUR CODE HERE

In [ ]:
# Select every 3 hours 

pass  # YOUR CODE HERE

##### Trust us and start from this already preprocessed dataset for this challenge

In [ ]:
url = "https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/weather_every_three_hours_engineered.csv"
df = pd.read_csv(url).drop(columns = ['Unnamed: 0'])
df

👆 In the preprocessed dataset, we have :
- $23$k rows  (~ 8 years of weather data)
- $19$ features composed of:
    - $1$ <font color=green>**target**</font> (we will use the past values of the temperature as a feature)
    - $18$ <font color=orange>**past covariates**</font> (= features which past values are known)
    - $0$ <font color=blue>**future covariates**</font> (= features which future values are known, e.g. public holidays)

    
<img src='https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/time-series-covariates.png'>

👨🏻‍🏫 This weather dataset is a DataFrame (dimension = 2) which is a single Time Series from the beginning of 2009 to the end 2016 with records every 3 hours. 

* `df.shape = (n_timesteps, n_features) = (23363, 19)`

🎯 The goal is to predict the temperature in 3, 6, 9, 12, ... hours using the past values.

In [ ]:
# Let's define the global variables of our dataset
TARGET = 'T (degC)'
N_TARGETS = 1
N_FEATURES = 19

### (0.0.2) Visualising your Time Series

📈  Here is the ***evolution of some features over time***:
* `T (degC)` (temperature)
* `p (mbar)` (atmospheric pressure)
* `rho (g/m**3)` (atmospheric density)



In [ ]:
plot_cols = [TARGET, 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = df.index
plot_features.plot(subplots = True);

In [ ]:
zoomed_slice = slice(1000,1100)

plot_features = df.loc[zoomed_slice, plot_cols]
plot_features.plot(subplots = True);

# (1) Prepare the dataset

##  (1.0) The big picture

<b><u>Step 1: Cross-Validation in Time Series [FOLDS] </u></b>

* Starting from this single Time Series, we will create <font color="#c91ac9">**FOLDS**</font>...
* ... and train/evaluate our LSTM on these different <font color="#c91ac9">**FOLDS**</font> to conclude about the robustness of the neural network
* It is very common to create ***hundreds*** of <font color="#c91ac9">**FOLDS**</font> in Time Series forecasting, in order to to cover all types of external conditions: e.g.
    - crash market periods 📉
    - bull markets 📈
    - atone markets 😴, etc...

<b><u>Step 2: Holdout method within each fold [TRAIN-TEST SPLIT]</u></b>

* For each <font color="#c91ac9">**FOLD**</font>, we will do a TRAIN-TEST SPLIT to:
    * <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set 
    * and <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set
    * Always split the train set **chronologically** before the test set!

👇 The first two steps can be summarized in the following image that contains 4 FOLDS:

<img src="https://bit.ly/3yLoa92" alt="Time Series Cross Validation" width="500" height="500">


<b><u>Step 3: Sampling SEQUENCES in both the train set and the test set</u></b>

In each <font color=blue>**train**</font> set and each <font color="#ff8005">**test**</font> set, we will create <font color=magenta>**random sequences**</font> as illustrated down below 👇:

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 



## (1.1) Creating  <font color="#c91ac9">**FOLDS**</font> for cross-validation

Each of them with shape `(FOLD_LENGTH, n_features)` 

🌐 Let's define some global variables that we will use for our tests everywhere in this notebook:

In [ ]:
# --------------------------------------------------- #
# Let's consider FOLDS with a length of 3 years       #
# (2 years will be used for train, 1 for test!)       #
# --------------------------------------------------- #

FOLD_LENGTH = 8*365 * 3 # every 3 hrs x 8 = 24h
                        # three years

# --------------------------------------------------- #
# Let's consider FOLDS starting every trimester       #
# --------------------------------------------------- #
    
FOLD_STRIDE = 8*91 # every 3 hrs x 8 = 24h
                   # 1 quarter = 91 days

# --------------------------------------------------- #
# Let's consider a train-test-split ratio of 2/3      #
# --------------------------------------------------- #

TRAIN_TEST_RATIO = 0.66

❓ **Question (<font color="#c91ac9">FOLDS</font>)** ❓

Code the function `get_folds` below that we will use to create folds  `folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)`

In [ ]:
def get_folds(
    df: pd.DataFrame, 
    fold_length: int,
    fold_stride: int) -> List[pd.DataFrame]:
    '''
    This function slides through the Time Series dataframe of shape (n_timesteps, n_features) to create folds
    - of equal `fold_length`
    - using `fold_stride` between each fold
    
    Returns a list of folds, each as a DataFrame
    '''
    pass  # YOUR CODE HERE

🧪 ***Test your code***

In [ ]:
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)

print(f'The function generated {len(folds)} folds.')
print(f'Each fold has a shape equal to {folds[0].shape}.')

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('folds',
                         number_of_folds = len(folds),
                         fold_shape = folds[0].shape)
result.write()
print(result.check())

☝️ This amount of <font color="#c91ac9">**FOLDS**</font> should be enough to cross-validate our model correctly !

## (1.2) Temporal <font color=blue>**Train**</font>/<font color="#ff8005">**Test**</font> Split

👩🏻‍🏫 Let's focus on one fold for the moment.

In [ ]:
fold = folds[0]
fold

☝️ We want to split this fold **chronologically** into:

- a <font color=blue>train</font> dataframe
- a <font color="#ff8005">test</font> dataframe

that will contain all the data we need to be able to sample many `(X_i, y_i)` pairs in each dataframe in a next step! 

The `INPUT_LENGTH` of each `X_i` is going to be equal to 2 weeks (it is quite a common period for weather forecasting)

In [ ]:
INPUT_LENGTH = 8 * 14 # records every 3 hours x 8 = 24 hours
                      # two weeks

❓ **Question (temporal <font color=blue>train</font>-<font color=orange>test</font> split)** ❓

Code the function `train_test_split` down below which:
- <i>(input)</i> given a `fold` (like above), a `train_test_ratio` (e.g 0.8) and an `input_length` (fixed)
- <i>(output)</i> a tuple of (`fold_train`, `fold_test`) dataframes


❗️Hints to avoid data leakage❗️ 

- `fold_train` last index will become the last `y_train` later on
- `fold_test` first index will be used to create the first `X_test`
- To avoid data leakage, `y_train_last` should be JUST before `y_test_first`

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/explanations_for_train_test_split_temporal.png" alt="train_test_split_temporal" width="500" height="500"> 

In [ ]:
def train_test_split(fold:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> Tuple[pd.DataFrame]:
    '''
    Returns a train dataframe and a test dataframe (fold_train, fold_test)
    from which one can sample (X,y) sequences.
    df_train should contain all the timesteps until round(train_test_ratio * len(fold))   
    '''
    pass  # YOUR CODE HERE

🧪 ***Test your code***

In [ ]:
(fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('holdout',
                         train_index_start = fold_train.index.start,
                         train_index_stop = fold_train.index.stop,
                         test_index_start = fold_test.index.start,
                         test_index_stop = fold_test.index.stop)
result.write()
print(result.check())

## (1.3) Create (X, y) sequences

Now that we have splitted our fold into a <font color="blue">train</font> set and a <font color="#ff8005">test</font> set, it is time to:
- 🏋 sample lots of sequences $(X_i, y_i)$ on which the model will be <font color="blue">trained</font>
- 👩🏻‍🏫 sample lots of sequences $(X_i, y_i)$ on which the model will be <font color="#ff8005">evaluated</font>

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="300" height="300"> 

🎯 Our goal is to create 3D-arrays `(X_train, y_train)` and `(X_test, y_test)` containing all the SEQUENCES we need from this fold:

* `X_train.shape = (n_samples_train, input_length, n_features)`
* `y_train.shape = (n_samples_train, output_length, n_targets)`

👉 Notice that we are now dealing with **3D arrays** instead of **2D DataFrames** time-series

<img src="https://bit.ly/3bOhKNj" alt="3d arrays time series" width="1200" height="800"> 

In [ ]:
# INPUT X
print(f'N_FEATURES = {N_FEATURES}')
print(f'INPUT_LENGTH = {INPUT_LENGTH}')

☝️ `X` is the **input** of our model. 
- It contains $19$ features: the past values of the <font color=green>**target**</font> + $18$ <font color=orange>**past-covariates**</font>
- Each sequence has a length equal to $112$ timesteps (=$2$ weeks)

In [ ]:
# TARGET Y
print(f'N_TARGETS = {N_TARGETS}')

# Let's only predict 1 value ahead of us
OUTPUT_LENGTH = 1

☝️ `y` is the <font color=green>**target**</font> that we want to predic:t
- It is the value at the single next timestep (=3 hours later)
- We could also predict `OUTPUT_LENGTH > 1`, i.e several values in the future (3 hours later, 6 hours later, 9 hours later, ...) but let's keep it simple here and just try to predict the next point (in 3 hours)

💡 To create these SEQUENCES within the <font color="blue">train</font> set and the <font color="#ff8005">test</font> set, you have several options, among them:
- 🎲 <u><i>Option 1</i></u>: creating these sequences by randomly sampling $(X_i, y_i)$ from <font color="blue">fold_train</font> and <font color="#ff8005">fold_test</font>.
- ⌚️ <u><i>Option 2</i></u>: scanning a fold chronologically and create all the possible pairs $(X_i, y_i)$. 
 

👉 Let's focus on the first option: random sampling

🎁 If you want to scan the folds chronologically, we provided the solution in the section and you can come back to it later


### (1.2.1) Option 1: Create (X, y) by random sampling in each split

👇 We will code:

* 1️⃣ a function `get_Xi_yi` to generate a single sequence randomly from within a fold

* 2️⃣ a function `get_X_y` to generate 3D-arrays containing multiple sequences from a fold, calling the first function `get_Xi_yi` many times over

#### (1.2.2.1) Generating one random sequence

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/get_xi_yi.png" alt="one sequence" width="400" height="400"> 

❓ **Question (extracting a random sequence from a fold)** ❓

Code the function `get_Xi_yi` down below which:
- <i>(input)</i> given a fold, an `input_length` and an `output_length`
- <i>(output)</i> returns a sequence $(X_i,y_i)$

In [ ]:
def get_Xi_yi(
    fold:pd.DataFrame, 
    input_length:int, 
    output_length:int):
    '''
    - given a fold, it returns one sequence (X_i, y_i)
    - with the starting point of the sequence being chosen at random
    '''
    pass  # YOUR CODE HERE

🧪 ***Test your code below***

In [ ]:
X_train_i, y_train_i = get_Xi_yi(fold_train, INPUT_LENGTH, OUTPUT_LENGTH)
X_test_i, y_test_i = get_Xi_yi(fold_test, INPUT_LENGTH, OUTPUT_LENGTH)

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('sequence',
                         x_train_i_shape = X_train_i.shape,
                         y_train_i_shape = y_train_i.shape,
                         x_test_i_shape = X_test_i.shape,
                         y_test_i_shape = y_test_i.shape)
result.write()
print(result.check())

In [ ]:
# Advanced check 😈 
# You should not allow some truncated (X_i,y_i) pairs to be generating,
# should you start sampling too close from the end of the fold... !

X_last, y_last = get_Xi_yi(fold_test, input_length=len(fold_test)-1, output_length=OUTPUT_LENGTH)
assert y_last.values == fold_test.iloc[-1,:][TARGET]

#### (1.2.2.2) Generating multiple random sequences

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 

❓ **Question (extracting multiple random sequence from a fold)** ❓

Code the function `get_X_y` down below which:
- <i>(input)</i> given a fold, a `number_of_sequences` an `input_length` and an `output_length`
- <i>(output)</i> returns $(X,y)$ 

_Don't forget to use the `get_Xi_yi` function that you have just coded!_

In [ ]:
def get_X_y(
    fold:pd.DataFrame,
    number_of_sequences:int,
    input_length:int,
    output_length:int
):
    pass  # YOUR CODE HERE

🧪 ***Test your code***

In [ ]:
N_TRAIN = 6666 # number_of_sequences_train
N_TEST =  3333 # number_of_sequences_test

X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('multiple_sequences',
                         x_train_shape = X_train.shape,
                         y_train_shape = y_train.shape,
                         x_test_shape = X_test.shape,
                         y_test_shape = y_test.shape)
result.write()
print(result.check())

### (1.2.2) (Don't read this now, keep for later) 🎁 Option 2: Scanning  chronologically

As stated earlier, there are multiple ways to extract sequences from a fold. 

- 🎲 In the previous section, you coded:
    - `get_Xi_yi` which randomly samples _one_ sequence 
    - and `get_X_y` which randomly generates _multiple_ sequences

- ⌚️ In this section, we provide you a unique function `get_X_y_strides`.
    - It scans a fold chronologically based on:
         - an `input_length` (let's still use `INPUT_LENGTH = 8 * 14`, i.e. two weeks) 
         - and a `sequence_stride` (think about a one-dimensional convolutional operation!)

👉 Let's scan the fold with a temporal stride of 3 hours (the minimum one):

In [ ]:
SEQUENCE_STRIDE = 1

In [ ]:
def get_X_y_strides(fold: pd.DataFrame, input_length: int, output_length: int, sequence_stride: int):
    '''
    - slides through a `fold` Time Series (2D array) to create sequences of equal
        * `input_length` for X,
        * `output_length` for y,
    using a temporal gap `sequence_stride` between each sequence
    - returns a list of sequences, each as a 2D-array time series
    '''

    X, y = [], []

    for i in range(0, len(fold), sequence_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (i + input_length + output_length) > len(fold):
            break
        X_i = fold.iloc[i:i + input_length, :]
        y_i = fold.iloc[i + input_length:i + input_length + output_length, :][[TARGET]]
        X.append(X_i)
        y.append(y_i)

    return np.array(X), np.array(y)

🧑🏻‍🎓 Some clarifications about scanning a fold sequentially :

In [ ]:
print("FOLD_LENGTH") 
print(f"= {FOLD_LENGTH} timesteps")
print(f"= {int(FOLD_LENGTH/8)} days") # 8 records per day, every 3 hours
print(f"= {int(FOLD_LENGTH/8/7)} weeks")

In [ ]:
X_train, y_train = get_X_y_strides(fold_train, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)
X_test, y_test = get_X_y_strides(fold_test, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)

print(X_train.shape)
print(y_train.shape)

👇 Illustration down below if *SEQUENCE_STRIDE = 1 week*

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/scanning_a_time_series_chronologically_v3.png">

# (2) Modelling

**The MAE as a metrics to monitor the temperature prediction**

The Mean Absolute Error seems to be a reasonable metrics to evaluate a model's capability to predict the temperature:

$$ MAE = \frac{1}{n_{samples}} \times \sum_{i = 1}^{n_{samples}} |y_{true}^{(i)} - y_{pred}^{(i)}|$$

## (2.1) A Recurrent Neural Network: the `LSTM`

🚀 It is time to design a Recurrent Neural Network and hopefully beat the baseline 💪 !

❓ **Question (RNN)** ❓ 

- Create a function `init_model` which builds and compiles a simple Recurrent Neural Network with an LSTM layer
- Don't forget to normalize your data with a [📚 Normalization layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

def init_model(X_train, y_train):
    
    pass  # YOUR CODE HERE

In [ ]:
model = init_model(X_train, y_train)
model.summary()

🛠 🎁 📉 We coded a function `plot_history` for you to visualize the training of your RNN over epochs. This function shows both the evolution of the loss function (MSE) and metrics (MAE).

In [ ]:
def plot_history(history):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:MAE ---
    
    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

❓ **Questions (training and evaluating)** ❓

- Code the `fit_model()` method that 
    - <i>(input)</i> given `model`
    - <i>(output)</i> returns the fitted model through a tuple `(model, history)` 
- Then:
    - Initialize an RNN model with the `init_model` function
    - <font color=blue>Train</font> the model
    - <font color=orange>Evaluate</font> it on the test set

In [ ]:
# YOUR CODE HERE

## (2.2) A Baseline Model to compare our LSTM with!

👉 In Time Series, an "intuitive" baseline model is to predict the **last seen value** for the future value(s) you want to forecast, as illustrated down below!

<img src = "https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/rnn_time_series_no_horizon.png" width = 600, height = 300>

❓ **Question (Last Seen Value Baseline Model)** ❓ 

Create a new method `init_baseline` that initializes and compiles a baseline model in a similar way to the LSTM.

- The baseline should be as a [📚 Keras Lambda Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda)
- It should not even need a `fit` because it should have 0 trainable params
- Then, compute the MAE for this FOLD and compare it with the MAE you obtained on the test set after training the LSTM 
 
<details>
    <summary markdown='span'>💡 <i>Hint</i></summary>

- Remember that `X_train[:,:,1]` is your temperature time-series (it's the second column in your `fold`) 
</details>


In [ ]:
from tensorflow.keras.layers import Lambda

def init_baseline():

    pass  # YOUR CODE HERE

In [ ]:
baseline_model = init_baseline()
baseline_score = baseline_model.evaluate(X_test, y_test)
print(f"- The Baseline MAE on the test set is equal to {round(baseline_score[1],2)} Celsius degrees")

In [ ]:
print(f"- The LSTM MAE on the test set is equal to {round(res[1],2)} Celsius degrees")
print(f"🔥 Improvement of the LSTM model over the baseline (on this fold for the test set) = : {round((1 - (res[1]/baseline_score[1]))*100,2)} % 🔥")

## (2.3) Cross-Validation

Remember **"The big picture"** ? 

<br>

<details>
    <summary markdown='span'><i>The big picture</i></summary>


<b><u>Step 1: Cross-Validation in Time Series [FOLDS] </u></b>

* Starting from this single Time Series, we will create <font color="#c91ac9">**FOLDS**</font>...
* ... and train/evaluate our LSTM on these different <font color="#c91ac9">**FOLDS**</font> to conclude about the robustness of the neural network
* It is very common to create ***hundreds*** of <font color="#c91ac9">**FOLDS**</font> in Time Series forecasting, so as to cover all type of external conditions: e.g.
    - crash market periods 📉
    - bull markets 📈
    - atone markets 😴, etc...

<b><u>Step 2: Holdout method within each fold [TRAIN-TEST SPLIT]</u></b>

* For each <font color="#c91ac9">**FOLD**</font>, we will do a TRAIN-TEST SPLIT to:
    * <font color=blue>**fit**</font> the model on the <font color=blue>**train**</font> set 
    * and <font color="#ff8005">**evaluate**</font> it on the <font color="#ff8005">**test**</font> set
    * Always split the train set **chronologically** before the test set!

👇 The first two steps can be summarized in the following image that contains 4 FOLDS:

<img src="https://bit.ly/3yLoa92" alt="Time Series Cross Validation" width="500" height="500">


<b><u>Step 3: Sampling SEQUENCES in both the train set and the test set</u></b>

In each <font color=blue>**train**</font> set and each <font color="#ff8005">**test**</font> set, we will create <font color=magenta>**random sequences**</font> as illustrated down below 👇:

<img src="https://bit.ly/3Ri8Vfd" alt="Sequences in each fold" width="500" height="500"> 


</details>

❗️ ***Warnings*** ❗️

* Keep in mind that we did <u><i>step 2</i></u> (<font color="blue">train</font>/<font color="orange">test</font> split) and <u><i>step 3</i></u> (get_X_y) only **<font color="#c91ac9">for one single FOLD</font>**. 
* ***If we want to ensure the robustness of a model, we need to <font color="">cross-validate the model on <font color="#c91ac9">ALL the folds</font>!***


In [ ]:
# Reminders of the global variables in this notebook

print(f'N_FEATURES = {N_FEATURES}') 
print(f'N_TARGETS = {N_TARGETS}') 
print('')
print(f'FOLD_LENGTH = {FOLD_LENGTH}')
print(f'FOLD_STRIDE = {FOLD_STRIDE}')
print(f'TRAIN_TEST_RATIO = {TRAIN_TEST_RATIO}')
print('')
print(f'N_TRAIN = {N_TRAIN}')
print(f'N_TEST = {N_TEST}')
print(f'INPUT_LENGTH = {INPUT_LENGTH}')
print(f'OUTPUT_LENGTH = {OUTPUT_LENGTH}')


In [ ]:
# Remember how many folds do we have ?
folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)

print(f"WARNING, we have {len(folds)} FOLDS, so you may want to run the cross-validation of the RNN on Colab.")

**❓ Cross validate your baseline and your model in a for-loop!❓**

Re-using your previously defined methods
- `get_folds`
- `train_test_split`
- `get_X_y`
- `init_model`
- `init_baseline`

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def cross_validate_baseline_and_lstm():
    '''
    This function cross-validates 
    - the "last seen value" baseline model
    - the RNN model
    '''
    
    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []
    
    # 0 - Creating folds
    # =========================================    
    folds = get_folds(df, FOLD_LENGTH, FOLD_STRIDE)
    
    for fold_id, fold in enumerate(folds):
        
        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_test) = train_test_split(fold, TRAIN_TEST_RATIO, INPUT_LENGTH)                   

        X_train, y_train = get_X_y(fold_train, N_TRAIN, INPUT_LENGTH, OUTPUT_LENGTH)
        X_test, y_test = get_X_y(fold_test, N_TEST, INPUT_LENGTH, OUTPUT_LENGTH)
        
        # 2 - Modelling
        # =========================================
        
        ##### Baseline Model
        baseline_model = init_baseline()
        mae_baseline = baseline_model.evaluate(X_test, y_test, verbose=0)[1]
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = init_model(X_train, y_train)
        es = EarlyStopping(monitor = "val_mae",
                           mode = "min",
                           patience = 2, 
                           restore_best_weights = True)
        history = model.fit(X_train, y_train,
                            validation_split = 0.3,
                            shuffle = False,
                            batch_size = 32,
                            epochs = 50,
                            callbacks = [es],
                            verbose = 0)
        res = model.evaluate(X_test, y_test, verbose=0)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")
        
        ##### Comparison LSTM vs Baseline for the current fold
        print(f"🏋🏽‍♂️ improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model

In [ ]:
%%time
# WARNING : it takes 75 minutes to run this cell 
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

In [ ]:
print(f"average percentage improvement over baseline = {round(np.mean(1 - (np.array(mae_lstms)/np.array(mae_baselines))),2)*100}%")

## Acknowledgments

* This challenge was truly inspired by the `Time Series Forecasting` tutorial from `Google>Tensorflow>Keras`
* The technical functions were inspired by Bruno Lajoie's [data-template package about Time Series](https://github.com/lewagon/data-templates/tree/main/project-boilerplates)